In [ ]:
!pip3 install torch
!pip3 install GPUtil
!pip3 install datasets
!pip3 install transformers
!pip3 install os
!pip3 install git-lfs

In [2]:
from GPUtil import showUtilization as gpu_usage
gpu_usage()  

| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
|  1 |  0% |  0% |
|  2 |  0% |  0% |
|  3 |  0% |  0% |


In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
%env CUDA_VISIBLE_DEVICES= 1

env: CUDA_VISIBLE_DEVICES=1


In [4]:
import numpy as np
import pandas as pd
import spacy
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
from transformers import DataCollatorWithPadding
from transformers import  TrainingArguments, Trainer
from datasets import load_metric
from datasets import load_dataset

2022-10-17 16:50:17.267941: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-17 16:50:34.497282: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-17 16:50:34.537106: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-17 16:51:00.981039: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [5]:
imdb = load_dataset("imdb")

Found cached dataset imdb (/home/cokeom/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)
tokenized_imdb = imdb.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

In [ ]:
# save_directory ='./sentimentModel'
# model.save_pretrained(save_directory)
# tokenizer.save_pretrained(save_directory)

In [13]:
from huggingface_hub import notebook_login

notebook_login()

In [18]:
model_name = 'distillbert-finetuned-imdb'
model.push_to_hub(model_name)
tokenizer.push_to_hub(model_name)

Configuration saved in /tmp/tmpscn41pjw/config.json
Model weights saved in /tmp/tmpscn41pjw/pytorch_model.bin
Uploading the following files to cokeoma/distillbert-finetuned-imdb: config.json,pytorch_model.bin
tokenizer config file saved in /tmp/tmp98p3kjuz/tokenizer_config.json
Special tokens file saved in /tmp/tmp98p3kjuz/special_tokens_map.json
Uploading the following files to cokeoma/distillbert-finetuned-imdb: tokenizer_config.json,special_tokens_map.json,vocab.txt,tokenizer.json


CommitInfo(commit_url='https://huggingface.co/cokeoma/distillbert-finetuned-imdb/commit/09647c5ebb2fa235a8be117dcafdab8bc39de504', commit_message='Upload tokenizer', commit_description='', oid='09647c5ebb2fa235a8be117dcafdab8bc39de504', pr_url=None, pr_revision=None, pr_num=None)